In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import nn
import pandas as pd
import numpy as np
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Bidirectional, MaxPool1D

2023-04-20 14:25:00.257127: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


# 데이터 로드

In [7]:
import pickle
with open('audio_augmented_per3500.pkl',"rb") as fr: #train 로드
    train = pickle.load(fr)
train

,index,Segment ID,mfcc_scaled,text_tokenize,temp+eda,sentiment_x
0,0,Sess13_script06_User026F_019,"[[0.0, 0.9999999999999999, 0.9608148846764442,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
1,1,Sess13_script06_User026F_019,"[[0.0, 0.8893497060147615, 0.9251501458319147,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
2,2,Sess13_script06_User026F_019,"[[0.0, 0.9672434656487046, 0.9493295501081099,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
3,3,Sess13_script06_User026F_019,"[[0.0, 0.8324701301043231, 0.8910219154763063,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
4,4,Sess13_script06_User026F_019,"[[0.0, 0.8692361742952204, 0.8943141198396264,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
...,...,...,...,...,...,...
24495,7379,Sess35_script04_User070F_008,"[[0.032945521331930516, 0.40808203802457316, 0...","[42, 262, 1, 2859, 326, 32, 2859, 100, 344, 11...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0
24496,7381,Sess31_script06_User061M_026,"[[0.0, 0.489686164522752, 0.8910410850529435, ...","[1819, 224, 248, 12, 1140, 0, 0, 0, 0, 0, 0, 0...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0
24497,7383,Sess34_script02_User068M_020,"[[0.0012534706618411162, 0.4571524437803839, 0...","[534, 3934, 3934, 46, 1377, 4, 1618, 0, 0, 0, ...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0
24498,7387,Sess27_script06_User054M_001,"[[0.09887480369794877, 0.5645720231907226, 0.7...","[15, 87, 722, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0


In [10]:
for i in range(0,len(train['mfcc_scaled'])):
    train['mfcc_scaled'][i]=np.mean(train['mfcc_scaled'][i],axis=1).reshape(600,1) #음성 데이터는 각 시퀀스끼리 mean 연산 진행

/tmp/ipykernel_1830638/2951617071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['mfcc_scaled'][i]=np.mean(train['mfcc_scaled'][i],axis=1).reshape(600,1)


In [2]:
import pickle
with open('valid.pkl',"rb") as fr: #valid 로드
    valid = pickle.load(fr)
valid=valid.reset_index()

In [11]:
for i in range(0,len(valid['mfcc_scaled'])):
    valid['mfcc_scaled'][i]=np.mean(valid['mfcc_scaled'][i],axis=1).reshape(600,1)

/tmp/ipykernel_1830638/3701760537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['mfcc_scaled'][i]=np.mean(valid['mfcc_scaled'][i],axis=1).reshape(600,1)


In [13]:
import pickle
with open('test.pkl',"rb") as fr: #test 데이터 로드
    test = pickle.load(fr)
test=test.reset_index()

In [14]:
for i in range(0,len(test['mfcc_scaled'])):
    test['mfcc_scaled'][i]=np.mean(test['mfcc_scaled'][i],axis=1).reshape(600,1)

/tmp/ipykernel_1830638/468462434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mfcc_scaled'][i]=np.mean(test['mfcc_scaled'][i],axis=1).reshape(600,1)


In [15]:
y_train=train['sentiment_x']

In [16]:
y_valid=valid['sentiment_x']

In [17]:
y_test=test['sentiment_x']

# 트랜스포머

In [65]:
class a_TokenAndPositionEmbedding(layers.Layer):#오디오 임베딩
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.a1_emb = tf.keras.layers.Conv1D(filters=32, kernel_size=5,padding='valid', activation='relu') #conv1d 연산 2번 진행
        self.a2_emb = tf.keras.layers.Conv1D(filters=32, kernel_size=5,padding='valid', activation='relu')
        self.a3_emb = MaxPool1D(pool_size=(12),padding='same') #maxpooling 진행
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim) 

    def call(self, x):
        maxlen = 50
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.a1_emb(tf.convert_to_tensor(x))
        x = self.a2_emb(x)
        x = self.a3_emb(x)
        
        return x + positions #포지셔널 임베딩과 음성 임베딩 값 더해주기

In [20]:


class TransformerBlock(layers.Layer):  #트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(embedding_dim=embed_dim, num_heads=num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        return self.layernorm2(out1 + ffn_output)

In [21]:
class MultiHeadAttention(tf.keras.layers.Layer): #트랜스포머 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=4):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim 
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [66]:
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen=50
training=1

audio_input = layers.Input(shape=(600,1)) 
a_embedding_layer = a_TokenAndPositionEmbedding(50, 32) #오디오 임베딩
audio_embedding = a_embedding_layer(audio_input)

transformer_block = TransformerBlock(32, num_heads, ff_dim) #오디오 트랜스포머 블럭

output = transformer_block(audio_embedding)

x = layers.GlobalAveragePooling1D()(output)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="sigmoid")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(7, activation="softmax")(x)  

model = keras.Model(inputs=audio_input, outputs=outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

# 모델 학습

In [67]:
history = model.fit(
    [np.array(train['mfcc_scaled'].to_list())], y_train, batch_size=16, epochs=5, validation_data=([np.array(valid['mfcc_scaled'].to_list())], y_valid)
) 

Epoch 1/5
1532/1532 [==============================] - 13s 7ms/step - loss: 1.7343 - accuracy: 0.2959 - val_loss: 0.6900 - val_accuracy: 0.8310
Epoch 2/5
1532/1532 [==============================] - 10s 6ms/step - loss: 1.5790 - accuracy: 0.3465 - val_loss: 0.7066 - val_accuracy: 0.8310
Epoch 3/5
1532/1532 [==============================] - 10s 6ms/step - loss: 1.5334 - accuracy: 0.3884 - val_loss: 0.7028 - val_accuracy: 0.8310
Epoch 4/5
1532/1532 [==============================] - 10s 6ms/step - loss: 1.3927 - accuracy: 0.4514 - val_loss: 0.7301 - val_accuracy: 0.8246
Epoch 5/5
1532/1532 [==============================] - 10s 6ms/step - loss: 1.2731 - accuracy: 0.4967 - val_loss: 0.6984 - val_accuracy: 0.8310


# 테스트 데이터셋으로 예측

In [68]:
y_pred=model.predict([np.array(test['mfcc_scaled'].to_list())])
yp=[]
for i in y_pred:
    yp.append(i.argmax())

# 모델 평가

In [69]:
from sklearn import metrics

print(metrics.classification_report(y_test.to_list(), yp, output_dict=True, digits=5)['weighted avg'])

{'precision': 0.6452463916397767, 'recall': 0.8032723022984029, 'f1-score': 0.7156394414946237, 'support': 2567}


/home/kie/anaconda3/envs/yajhmulmo/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kie/anaconda3/envs/yajhmulmo/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kie/anaconda3/envs/yajhmulmo/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

증강 전 데이터를 사용하면 오버피팅이 매우 크고, 모델이 중립 감정만 찍는 경향이 있음 

- 증강을 통해 모든 감정에 대해 모델이 잘 식별할 수 있도록 학습을 하고, sklearn 의 metrics를 활용하여 모든 감정에 대해 f1score를 구할 수 있었음

- test 데이터셋은 랜덤 8개의 세션으로 구성된 데이터 셋임
- test 데이터셋의 감정 분포도 중립이 타 감정(ex: 행복)에 비해 매우 많으므로 macro score가 아닌, weigthed score로 평가 진행
- 최종스코어는 아래와 같음
- {'precision': 0.6452463916397767, 'recall': 0.8032723022984029, 'f1-score': 0.7156394414946237} 의 성능을 낼 수 있었음.